<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Naive Bayes


---

## Learning Objectives

- Describe Naive Bayes
- Choose a Naive Bayes implementation based on your use case
- Implement a Naive Bayes model through scikit-learn

<h1>Lesson Guide<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Learning-Objectives" data-toc-modified-id="Learning-Objectives-1">Learning Objectives</a></span></li><li><span><a href="#Introduction" data-toc-modified-id="Introduction-2">Introduction</a></span></li><li><span><a href="#A-Simple-Spam-Example" data-toc-modified-id="A-Simple-Spam-Example-3">A Simple Spam Example</a></span><ul class="toc-item"><li><span><a href="#Problem:-Multiple-Features-Require-Joint-Probabilities" data-toc-modified-id="Problem:-Multiple-Features-Require-Joint-Probabilities-3.1">Problem: Multiple Features Require Joint Probabilities</a></span></li><li><span><a href="#Solution:-The-Naive-Bayes-Independence-Assumption" data-toc-modified-id="Solution:-The-Naive-Bayes-Independence-Assumption-3.2">Solution: The Naive Bayes Independence Assumption</a></span></li><li><span><a href="#Spam-Application:-Multiple-Features" data-toc-modified-id="Spam-Application:-Multiple-Features-3.3">Spam Application: Multiple Features</a></span></li></ul></li><li><span><a href="#Production-Issues" data-toc-modified-id="Production-Issues-4">Production Issues</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-5">Summary</a></span></li><li><span><a href="#Implementation-in-Scikit-Learn" data-toc-modified-id="Implementation-in-Scikit-Learn-6">Implementation in Scikit-Learn</a></span></li><li><span><a href="#Discriminative-and-Generative-Models" data-toc-modified-id="Discriminative-and-Generative-Models-7">Discriminative and Generative Models</a></span></li><li><span><a href="#Additional-Resources" data-toc-modified-id="Additional-Resources-8">Additional Resources</a></span></li></ul></div>

## Introduction

Naive Bayes is a classification algorithm relying on Bayes' rule to assign class labels $y$ for given feature values $X$. As in all other classification models, one is interested in determining the probability of having a certain class label when a certain combination of feature values is obtained

$$P(y|X)$$



Using Bayes' rule we can write this as

$$
P(y|X) = 
\frac{P(X|y)P(y)}
{P(X)}
$$

which is saying that if we know how likely a certain combination of feature values is within a particular class, if we know how likely that class is to occur, and if we know how likely that combination of feature values is across all classes, we also know how likely the class is given the feature values.

That is all we need! If we have enough data with examples for all possible feature values, we just have to look up the respective frequencies in the data and we are done.

## A Simple Spam Example

Suppose we are trying to predict spam emails. For now, we have one feature: whether or not the email mentions "guarantee." All we need to do is to look up what is the fraction of emails containing this word, the fraction of spam emails, and the fraction of spam emails which do contain that word.

$G$ = Guarantee, $S$ = Is spam.

 $$P\left(S|G\right) = \frac{P\left(G|S\right)P\left(S\right)}{P(G)} = \frac{P\left(G|S\right)P\left(S\right)}{P(G|S)P(S) + P(G|\neg{S})P(\neg{S})}$$


The denominator looks complicated, but it actually isn't. Because $G$ is binary (either present or not), then:

> $$P(G) = P(G,S) + P(G,\neg{S})=P(G|S)P(S) + P(G|\neg{S})P(\neg{S})$$

### Problem: Multiple Features Require Joint Probabilities

In this spam example, we only had one feature: $G$. But, in general, we'll probably use more than one. Really, we want to see some feature vector $X_1, X_2, ..., X_n$ (standing for all the words contained in the email):

$$P\left(S|X_1,\ldots, X_n\right) = \frac{P\left(X_1,\ldots, X_n|S\right)P(S)}{P(X_1,  \ldots, X_n|S)P(S) + P(X_1, \ldots, X_n|\neg{S})P(\neg S)}$$

For example, what is the likelihood that an email is spam given that the email mentions "guarantee," "oil," "prince," and "million," but not "meeting", "colleague," and "dad?"

With a lot of features, calculating the joint probabilities quickly becomes complicated. We would definitely need a lot of data to ensure we have enough feature combinations. 
If we were interested in the presence or absence of only a 100 selected words, we would have $2^{100}\approx 10^{30}$ possible feature combinations, and we need to estimate the probability for each of them occurring. The number of possible combinations grows exponentially with the number of features, and we are even just talking about presence or absence.

No matter how diligent we are, we may never collect a single training example that contains the precise combination of feature words we need. Therefore, we would be unable to classify a new email containing a particular combination of words.

Although each probability is easy to calculate, the joint probability model is simply not practical.

### Solution: The Naive Bayes Independence Assumption

We're stuck, because conditional joint probabilities are required for multiple features. This means there are exponentially more probabilities to compute and data to collect.

To get around this, let's make an assumption: 

> **All $X_i$ are conditionally independent given $S$** 

(where $S$ indicates "is spam"). This means that, given $S$, no two $X_i$ depend on each other. For example, the words "million" and "prince" each independently indicate that an email is spam. So, it's not the complex interaction of words that determines spam; each feature can indicate whether or not an email is spam independently.


Recall that if events $A$ and $B$ are independent, then the joint probability is simply 

$$P(A,B) = P(A)P(B)$$ 

Similarly, if $A$ and $B$ are conditionally independent of $S$, then 

$$P(A,B|S) = P(A|S)P(B|S)$$

This formula works out well in general, too:

$$P\left(X_{1}X_{2} \dots X_{n}|S\right) = P\left(X_{1} |S\right)  P\left(X_{2} |S\right) \cdots P\left(X_{n} |S\right)$$

> **Caution:** Of course, this assumption is rarely (if ever) true. Often, it requires precise reading to tell whether or not an email was written by a native speaker. In this case, it's often not the particular words used, but how they are used in context.


If we were considering how indicative the words "guarantee" and "million" are for being spam, we would write the following

$G$ = Guarantee, $M$ = Million, $S$ = Is spam:

$$P(S,G,M) = P(G,M|S)P(S) = P(G|S)P(M|S)P(S)$$


> None of these probabilities require us to examine multiple features at once in our data set, making them drastically easier to compute. 

In reality, model parameters/coefficients are unlikely to be independent. But Naive Bayes makes exactly this assumption and often works well despite this.

### Spam Application: Multiple Features

How is this used in practice? Let's combine the Naive Bayes simplification above with our original formula (the denominator is computed the same way as before, combined with our naive assumption):

$$
\begin{eqnarray*}
P\left(S|G,M\right) &=& \frac{P(S,G,M)}{P(G,M)} \\
 &=& \frac{P\left(G,M|S\right)P\left(S\right)}{P(G,M|S)P(S) + P(G,M|\neg{S})P(\neg{S})} \\
 &=& \frac{P\left(G|S\right)P\left(M|S\right)P\left(S\right)}{P(G|S)P(M|S)P(S) + P(G|\neg{S})P(M|\neg{S})P(\neg{S})}
\end{eqnarray*} 
$$
 
Typically, we compute this probability for each class (in this case, just $S$ or $\neg S$), then predict the class with the highest probability. Note that, for all of these, the denominator $P(G,M)$ is constant. Hence, this formula is often written as "proportional" ($\propto$), considerably simplifying it. Instead of comparing the exact probabilities, we can simply see how they score relative to each other. So:

 $$P\left(S|GM\right) \propto P\left(G|S\right)P\left(M|S\right)P\left(S\right)$$

## Production Issues

Accidentally assuming a zero probability for any word appearing in a given class could present major problems — the entire probability estimation would be zero.

- **New features:** What if a particular feature was never seen in our training data? Instead of using a zero probability, we should use a technique such as [Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) to estimate a small non-zero probability.

- **Underflow:** Probabilities could be very small if some features rarely occur in some classes. Recall that floating point often gives us trouble because of its limited precision — small floats tend toward zero. We can approach this problem by storing the logarithm of each probability, $P_i$, instead of $P_i$ itself:

$$\log(P_1P_2) = \log\ P_1 + \log\ P_2$$

$$e^{\log\ P_1} = P_1$$

So: $$P_1P_2 \cdots P_n = e^{\log\ P_1 + \ldots + \log\ P_n}$$

## Summary

**Why is the Naive Bayes formula important?** With the independence assumption, we don't need to compute every joint probability distribution, we only need to compute the probability of each word separately: $P(G|S)$ and $P(M|S)$. 

These calculations can be performed quickly using our training data. The downside is that if spam is actually determined by some complex interaction between "guarantee" and "millions" (e.g., one of the words on its own is not a good sign of spam, but both in the same email is), then the independence assumption does not hold and our model will not have the capacity to predict spam correctly.

To make Naive Bayes a classifier, all we have to do is compute the probability of $P(y|X)$ for each class, $y$. In math notation, this is:

$$ P(y \;|\; X_1, \ldots, X_n) \propto P(y) \prod_{i=1}^n P(X_i \;|\; y) \\
\downarrow \\
\hat{y} = {\rm arg} \; \underset{y}{\rm max} \; P(y) \prod_{i=1}^n P(X_i \;|\; y)$$

> The last expression simply means that we predict the class label with the highest predicted probability.

## Implementation in Scikit-Learn

- [Docs Bernoulli NB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html)
- [Docs Multinomial NB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)
- [Docs Gaussian NB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)

<img src="./images/naive-bayes.png">

The differences can be summarized as:
-    **BernoulliNB** is designed for binary/Boolean features.
-    The **Multinomial Naive Bayes classifier** is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as `tf-idf` may also work.
-    **GaussianNB** is designed for continuous, normally distributed features.

In [1]:
pwd

'/Users/paxton615/GA/DSI9-lessons/week09/day3_bayes_rule_and_naive_bayes/naive-bayes-lesson'

In [2]:
import numpy as np
import pandas as pd
from sklearn import naive_bayes
from sklearn.model_selection import cross_val_score

In [3]:
data = pd.read_csv('../../../../resource-datasets/spam/spam_words_wide.csv')
data.head()

,is_spam,getzed,86021,babies,sunoco,ultimately,thk,voted,spatula,fiend,...,itna,borin,thoughts,iccha,videochat,freefone,pist,reformat,strict,69698
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


> Which scikit-learn Naive Bayes implementation should we use?

In [4]:
target = data.pop('is_spam')

In [5]:
model = naive_bayes.BernoulliNB(binarize=None)
model.fit(data, target)

BernoulliNB(alpha=1.0, binarize=None, class_prior=None, fit_prior=True)

In [6]:
print(cross_val_score(model, data, target, cv=5).mean())

0.9373670608883271


> **Check**: Is that good?

In [7]:
1-target.mean()

0.8659368269921034

## Discriminative and Generative Models

- Naive Bayes is often called a Generative Model whereas Logistic Regression is a discriminative model.
- In a discriminative model we try to model $P(y|X)$ directly, we are only interested in how likely the class is given any combination of feature values. That is sufficient to obtain the decision boundary in feature space where both classes are equally likely.
- In a generative model, we obtain full access to the joint distribution of $P(y,X)$. From this joint distribution we could then sample to obtain new possibly observable class and feature combinations in the appropriate proportions learned from the data.

## Additional Resources

- [An interesting slide](https://web.stanford.edu/class/cs124/lec/naivebayes.pdf) from a Stanford MOOC that has a section on Naive Bayes
- [A much more technical paper](https://www.cs.cmu.edu/~tom/mlbook/NBayesLogReg.pdf) comparing Naive Bayes to logistic regression
- [More exposition on Naive Bayes](http://blog.yhat.com/posts/naive-bayes-in-python.html)
- [Naive Bayes from scratch](http://machinelearningmastery.com/naive-bayes-classifier-scratch-python/)
- [Mixing different variable types in Naive Bayes](https://sebastianraschka.com/faq/docs/naive-bayes-vartypes.html)